In [ ]:
import sys
sys.path.insert(0, '..')

from load_paus_cat import load_paus_cat

import matplotlib
import matplotlib.pyplot as plt
matplotlib.rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
matplotlib.rc('text', usetex=True)
matplotlib.rcParams.update({'font.size': 16})

import numpy as np

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.io import fits
from astropy.cosmology import Planck18 as cosmo

In [ ]:
# Load PAUS cat
field_name = 'W3'
cats_dir = '/home/alberto/almacen/PAUS_data/catalogs'
path_to_cat_list = [f'{cats_dir}/PAUS_3arcsec_{field_name}.csv',]
paus_cat = load_paus_cat(path_to_cat_list)

In [ ]:
# Load SDSS DR16Q catalog
sdss_dir = '/home/alberto/almacen/SDSS_spectra_fits'

path_to_sdss_cat = f'{sdss_dir}/DR16/DR16Q_Superset_v3.fits'
sdss_cat = fits.open(path_to_sdss_cat)[1].data

# And the line catalog
fread = fits.open(f'{sdss_dir}/spAllLine-v5_13_0.fits')
sdss_line_cat = fread[1].data[np.where(fread[1].data['LINEWAVE'] == 1215.67)]

In [ ]:
# Xmatch
coords_paus = SkyCoord(ra=paus_cat['RA'] * u.deg,
                       dec=paus_cat['DEC'] * u.deg)
coords_sdss = SkyCoord(ra=sdss_cat['RA'] * u.deg,
                       dec=sdss_cat['DEC'] * u.deg)
                    
xm_id, ang_dist, _= coords_paus.match_to_catalog_sky(coords_sdss)

# Objects with 1 arcsec of separation
mask_dist = (ang_dist <= 1 * u.arcsec)

In [ ]:
sdss_line_cat.columns

In [ ]:
F_lya = np.ones(len(paus_cat['RA'])).astype(float) * -1
F_lya_err = np.ones(len(paus_cat['RA'])).astype(float) * -1
EW_lya = np.ones(len(paus_cat['RA'])).astype(float) * -1
EW_lya_err = np.ones(len(paus_cat['RA'])).astype(float) * -1

k = 0
for src in np.where(mask_dist)[0]:
    fiber = sdss_cat['FIBERID'][xm_id[src]]
    mjd = sdss_cat['MJD'][xm_id[src]]
    plate = sdss_cat['PLATE'][xm_id[src]]

    where_mjd = np.where(sdss_line_cat['MJD'] == mjd)
    where_mjd_pl = np.where(sdss_line_cat['PLATE'][where_mjd] == plate)
    where_mjd_pl_fi = np.where(sdss_line_cat['FIBERID'][where_mjd[0][where_mjd_pl]] == fiber)
    where_in_line_cat = where_mjd[0][where_mjd_pl[0][where_mjd_pl_fi]]

    if len(where_in_line_cat) > 0:
        F_lya[src] = sdss_line_cat['LINEAREA'][where_in_line_cat]
        F_lya_err[src] = sdss_line_cat['LINEAREA_ERR'][where_in_line_cat]
        EW_lya[src] = sdss_line_cat['LINEEW'][where_in_line_cat]
        EW_lya_err[src] = sdss_line_cat['LINEEW_ERR'][where_in_line_cat]
    else:
        k += 1
        print(k)

In [ ]:
Dl = cosmo.luminosity_distance(sdss_cat['Z_CIV'][xm_id]).to(u.cm).value
L_lya = np.ones_like(F_lya) * -1
L_lya_err = np.ones_like(F_lya) * -1
L_lya_err_up = np.ones_like(F_lya) * -1
L_lya_err_down = np.ones_like(F_lya) * -1

L_lya[mask_dist] = np.log10(F_lya[mask_dist] * 1e-17 * 4 * np.pi * Dl[mask_dist]**2)
L_lya_err[mask_dist] = F_lya_err[mask_dist] * 1e-17 * 4 * np.pi * Dl[mask_dist] ** 2
L_lya_err_up[mask_dist] = np.log10(10**L_lya[mask_dist] + L_lya_err[mask_dist]) - L_lya[mask_dist]
L_lya_err_down[mask_dist] = L_lya[mask_dist] - np.log10(10**L_lya[mask_dist] - L_lya_err[mask_dist])

L_lya[~np.isfinite(L_lya)] = -1.
L_lya_err_up[~np.isfinite(L_lya)] = -1.
L_lya_err_up[~np.isfinite(L_lya)] = -1.

In [ ]:
# from LAE_selection_method import select_LAEs
# from PAUS_Lya_LF_corrections import L_lya_bias_apply

# nb_min, nb_max = 0, 20
# r_min, r_max = 17, 24

# paus_cat = select_LAEs(paus_cat, nb_min, nb_max, r_min, r_max)
# # paus_cat = L_lya_bias_apply(paus_cat, field_name, nb_min, nb_max)

# zspec = sdss_cat[xm_id]['Z_CIV']
# zspec[~mask_dist] = -1

In [ ]:
# w0_CIV = 1549.48
# w0_CIII = 1908.734

# w_CIV_measured = w0_CIV * (sdss_cat['Z_LYA'] + 1)
# w_CIII_measured = w0_CIII * (sdss_cat['Z_LYA'] + 1)

# w_CIV_obs = w0_CIV * (sdss_cat['Z_CIV'] + 1)
# w_CIII_obs = w0_CIII * (sdss_cat['Z_CIII'] + 1)

# mask_CIV = (sdss_cat['Z_CIV'] > 0)
# mask_CIII = (sdss_cat['Z_CIII'] > 0)
# mask_Lya = (sdss_cat['Z_LYA'] > 0)

# fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(10, 4), sharey=True)
# fig.subplots_adjust(wspace=0.03)

# bins = np.linspace(-100, 500, 50)
# ax0.hist((w_CIV_measured - w_CIV_obs)[mask_CIV],
#         bins, histtype='step')
# ax0.hist((w_CIII_measured - w_CIII_obs)[mask_CIII],
#         bins, histtype='step')

# bins = np.linspace(-100, 500, 50)
# ax0.hist((w_CIV_measured - w_CIV_obs)[mask_CIV],
#          bins, histtype='step', color='C0', label='CIV')
# ax0.hist((w_CIII_measured - w_CIII_obs)[mask_CIII],
#          bins, histtype='step', color='C1', label='CIII')

# bins = np.linspace(-0.1, 0.3, 50)
# ax1.hist((sdss_cat['Z_LYA'] - sdss_cat['Z_CIV'])[mask_CIV & mask_Lya],
#          bins, histtype='step', color='C0', label='CIV')
# ax1.hist((sdss_cat['Z_LYA'] - sdss_cat['Z_CIII'])[mask_CIII & mask_Lya],
#          bins, histtype='step', color='C1', label='CIII')

# CIV_w_percentiles = np.percentile((w_CIV_measured - w_CIV_obs)[mask_CIV & mask_Lya],
#                                   [1, 99])
# CIII_w_percentiles = np.percentile((w_CIII_measured - w_CIII_obs)[mask_CIII & mask_Lya],
#                                   [1, 99])
# print(f'{CIV_w_percentiles=}')
# ax0.axvline(CIV_w_percentiles[0], c='r', ls='--')
# ax0.axvline(CIV_w_percentiles[1], c='r', ls='--')
# print(f'{CIII_w_percentiles=}')
# ax0.axvline(CIII_w_percentiles[0], c='orange', ls=':')
# ax0.axvline(CIII_w_percentiles[1], c='orange', ls=':')

# ax0.legend()

# ax0.set_xlabel(r'$\Delta w_\mathrm{obs}$')
# ax1.set_xlabel(r'$\Delta z$')

# plt.show()

In [ ]:
EW_lya_err[EW_lya > 0]

In [ ]:
import pandas as pd

z_CIV = sdss_cat[xm_id]['Z_CIV']
z_CIII = sdss_cat[xm_id]['Z_CIII']
z_Lya = sdss_cat[xm_id]['Z_LYA']
z_CIV[~mask_dist] = -1

sdss_xmatch_to_save = {
    'ref_id': paus_cat['ref_id'],
    'L_lya': L_lya,
    'L_lya_err_up': L_lya_err_up,
    'L_lya_err_down': L_lya_err_down,
    'EW0_lya': EW_lya / (1 + z_CIV),
    'EW0_lya_err': EW_lya_err / (1 + z_CIV),
    'z_CIV': z_CIV,
    'z_CIII': z_CIII,
    'z_Lya': z_Lya,
    'NB_number': paus_cat['NB_number']
}
pd.DataFrame(sdss_xmatch_to_save).to_csv('/home/alberto/almacen/PAUS_data/catalogs/Xmatch_SDSS_W3.csv')

In [ ]:
# from paus_utils import z_NB

# fig, ax = plt.subplots(figsize=(6, 6))

# nb_min, nb_max = 0, 20

# z_min, z_max = z_NB(nb_min), z_NB(nb_max)

# zspec_mask = (zspec >= z_min) & (zspec <= z_max)
# ax.scatter(L_lya[zspec_mask], paus_cat['L_lya'][zspec_mask], label='Direct measurement')
# ax.scatter(L_lya[zspec_mask], paus_cat['L_lya_corr'][zspec_mask], label='Corrected')
# mask = zspec_mask & (L_lya > 40) & (paus_cat['L_lya_corr'] > 40) & (L_lya < 50) & (paus_cat['L_lya_corr'] < 50)
# print(np.nanmedian(L_lya[mask] - paus_cat['L_lya_corr'][mask]))
# print(np.nanstd(L_lya[mask] - paus_cat['L_lya_corr'][mask]))
# print(np.nanmedian(L_lya[mask] - paus_cat['L_lya'][mask]))
# print(np.nanstd(L_lya[mask] - paus_cat['L_lya'][mask]))

# xx = [-100, 100]
# ax.plot(xx, xx, ls='--', c='r')

# ax.legend(fontsize=18)

# ax.set(xlim=(43, 46), ylim=(43, 46))

# ax.set_xlabel(r'Spectroscopic $\log_{10}(L_{\mathrm{Ly}\alpha}/\mathrm{erg\,s}^{-1})$')
# ax.set_ylabel(r'Measured $\log_{10}(L_{\mathrm{Ly}\alpha}/\mathrm{erg\,s}^{-1})$')

# plt.show()

In [ ]:
from jpasLAEs.utils import bin_centers
from paus_utils import z_NB

nb_min, nb_max = 8, 10

z_min, z_max = z_NB(nb_min), z_NB(nb_max)

zphot = paus_cat['z_NB']
zspec_mask = (zspec >= z_min) & (zspec <= z_max)
zphot_mask = (zphot >= z_min) & (zphot <= z_max)
nice_z = np.abs(zspec - zphot) < 0.12

# Completeness SDSS based
L_bins = np.linspace(41, 46, 20)
L_bins_c = bin_centers(L_bins)
h_sdss_qso, _ = np.histogram(L_lya[zspec_mask & (paus_cat['NB_number'] > 39)], L_bins)
h_sel, _ = np.histogram(L_lya[paus_cat['nice_lya'] & zphot_mask
                              & (paus_cat['NB_number'] > 39)
                              & nice_z], L_bins)
h_sel_bad, _ = np.histogram(L_lya[paus_cat['nice_lya'] & zphot_mask
                              & (paus_cat['NB_number'] > 39)
                              & ~nice_z], L_bins)

fig, ax = plt.subplots(figsize=(6, 4))

ax.plot(L_bins_c, h_sel / h_sdss_qso,
        drawstyle='steps-mid', label='"Completeness"')
ax.plot(L_bins_c, h_sel / (h_sel + h_sel_bad),
        drawstyle='steps-mid', label='"Purity"')

ax.set_ylim(0, 1.01)
ax.legend(fontsize=12)

plt.show()


###############################################
###############################################

fig, ax = plt.subplots(figsize=(6, 4))

ax.plot(L_bins_c, h_sel, label='Selected',
        drawstyle='steps-mid')
ax.plot(L_bins_c, h_sdss_qso, label='All (SDSS)',
        drawstyle='steps-mid')

ax.legend(fontsize=11)

plt.show()

In [ ]:
sdss_Flya_all = sdss_line_cat['LINEAREA'] * 1e-17
sdss_Dl_arr = cosmo.luminosity_distance(sdss_line_cat['LINEZ']).to(u.cm).value
sdss_L_lya_all = np.log10(sdss_Flya_all * 4 * np.pi * sdss_Dl_arr**2)


In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))

ax.hist(sdss_L_lya_all, L_bins,
        histtype='step', density=True,
        label='All (SDSS)')
ax.hist(L_lya, L_bins, label='X-match (SDSS)',
        histtype='step', density=True)

ax.set_ylabel('\# sources [A. U.]')
ax.set_xlabel(r'$\log_{10}[L_{\mathrm{Ly}\alpha}$ / erg\,s$^{-1}$\,cm$^{-2}$\AA$^{-1}$]')

ax.legend(fontsize=12)
ax.set_yscale('log')

plt.show()